In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import datetime
from pyspark.sql.functions import col, lit, avg
from pyspark.sql import *

In [7]:
#importing csv file from hdfs
user_data= sc.textFile("hdfs://localhost:54310/user/hive/warehouse/data/user_info.csv").map(lambda x:x.split(","))

In [8]:
user_df = sqlContext.createDataFrame(data=user_data.filter(lambda x:x[0]!='user_name'),
                             schema=user_data.filter(lambda x:x[0]=='user_name').collect()[0])

In [9]:
# creating new column named seconds with username
hours_list = []
for row in user_df.collect():
    # converting string type of datetime to datetime
    date_time = datetime.datetime.strptime(row['working_hours'], "%Y-%m-%d %H:%M:%S")
    seconds_data = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    hours_list.append((row['user_name'], seconds_data))

In [13]:
 #creating new column dataframe to join with original dataframe( new_log_dataframe)
column = sqlContext.createDataFrame(hours_list, ('username',"seconds"))

In [14]:
#merge new column with the dataframe 
dataframe = (user_df
.join(column, col("user_name")==col("username")))


In [15]:
#obtaining avarage value of seconds by using aggregate function
average_data=dataframe.agg(avg(col("seconds")))
average = average_data.collect()[0][0]

In [16]:
#filtering dataframe with highest average hours users
highest_average_hours = dataframe.filter(dataframe['seconds'] > average)


In [17]:
highest_average_hours.select("user_name","seconds").show()

+--------------------+-------+
|           user_name|seconds|
+--------------------+-------+
|ashutoshrit64@gma...|  37501|
|giridhardandikwar...|  27900|
|salinabodale73@gm...|  27000|
|indrajeetgajbhiye...|  31800|
|khairnarswapna99@...|  33900|
|aman.raj251@gmail...|  26100|
|Krushnanikam26@gm...|  27601|
|magadum.iranna@gm...|  31199|
|puruissimple@gmai...|  26399|
|prajktasraut16@gm...|  32699|
|samruddhichitnis0...|  33908|
|surajwarbhuvan192...|  32400|
| grmule018@gmail.com|  29100|
|  shelkeva@gmail.com|  33001|
|akshaypatwari24@g...|  32700|
|rohitsarate1027@g...|  35401|
|sharlawar77@gmail...|  34200|
|kaleshwetaanil@gm...|  34199|
|rahilstar11@gmail...|  32100|
|antonyalexcm@gmai...|  36900|
+--------------------+-------+
only showing top 20 rows



In [18]:
#filtering users having highest average value
highest_average_hours.select("user_name").show()

+--------------------+
|           user_name|
+--------------------+
|ashutoshrit64@gma...|
|giridhardandikwar...|
|salinabodale73@gm...|
|indrajeetgajbhiye...|
|khairnarswapna99@...|
|aman.raj251@gmail...|
|Krushnanikam26@gm...|
|magadum.iranna@gm...|
|puruissimple@gmai...|
|prajktasraut16@gm...|
|samruddhichitnis0...|
|surajwarbhuvan192...|
| grmule018@gmail.com|
|  shelkeva@gmail.com|
|akshaypatwari24@g...|
|rohitsarate1027@g...|
|sharlawar77@gmail...|
|kaleshwetaanil@gm...|
|rahilstar11@gmail...|
|antonyalexcm@gmai...|
+--------------------+
only showing top 20 rows

